In [18]:
supply = {'texas City' : 2000, 'san antonio' : 2000}
hub = {'Los Angeles' : 1000, 'Philadelphia' : 1000, 'Phoenix' : 2000}
plants={'Austin' : 1500, 'Baltimore' : 1300, 'Charlotte' : 500, 'Chicago' : 400,
           'Columbus' : 300}

In [19]:
# test the supply dictionary
for i in supply.keys():
    print("The supply at",i,"is",supply[i])

The supply at texas City is 2000
The supply at san antonio is 2000


In [20]:
cost1= {( 'texas City', 'Los Angeles') : 203,
    ( 'texas City', 'Philadelphia') : 205,
    ( 'texas City', 'Phoenix') : 388,
    ( 'san antonio', 'Los Angeles') : 444,
    ( 'san antonio', 'Philadelphia') : 467,
    ( 'san antonio', 'Phoenix') : 467,
}

In [21]:
cost2={('Los Angeles', 'Austin') : 740,
    ( 'Los Angeles', 'Baltimore') : 794,
    ( 'Los Angeles', 'Charlotte') : 916,
    ( 'Los Angeles', 'Chicago') : 959,
    ( 'Los Angeles', 'Columbus') : 1048,
    ( 'Philadelphia', 'Austin') : 1093,
    ( 'Philadelphia', 'Baltimore') : 1194,
    ( 'Philadelphia', 'Charlotte') : 1312,
    ( 'Philadelphia', 'Chicago') : 1319,
    ( 'Philadelphia', 'Columbus') : 1329,
    ( 'Phoenix', 'Austin') : 1384,
    ( 'Phoenix', 'Baltimore') : 1448,
    ( 'Phoenix', 'Charlotte') : 1585,
    ( 'Phoenix', 'Chicago') : 1626,
    ( 'Phoenix', 'Columbus') : 781}


In [22]:
# define the ship_froms and ship_tos
ship_froms = [ key for key in supply.keys() ]
print("Ship-froms are",ship_froms)

ship_tos = [ key for key in hub.keys() ]
print("Ship-tos are",ship_tos)

ship_to2s = [ key for key in plants.keys() ]
print("Ship-to2s are",ship_to2s)

Ship-froms are ['texas City', 'san antonio']
Ship-tos are ['Los Angeles', 'Philadelphia', 'Phoenix']
Ship-to2s are ['Austin', 'Baltimore', 'Charlotte', 'Chicago', 'Columbus']


In [23]:
# import gurobi
!pip install gurobipy
import gurobipy as gp
from gurobipy import GRB

In [24]:
# create model object
m = gp.Model()

# create transportation variables
x = m.addVars( ship_froms, ship_tos )
y = m.addVars( ship_tos, ship_to2s )

In [25]:
# set objective function (minimize transportation cost)
m.setObjective( gp.quicksum(cost1[i,j] * x[i,j] for i,j in cost1.keys())
              + gp.quicksum(cost2[i,j] * y[i,j] for i,j in cost2.keys()) , GRB.MINIMIZE )

In [26]:
# add supply constraints
m.addConstrs( gp.quicksum( x[i,j] for j in ship_tos ) <= supply[i] for i in ship_froms )

{'texas City': <gurobi.Constr *Awaiting Model Update*>,
 'san antonio': <gurobi.Constr *Awaiting Model Update*>}

In [27]:
m.addConstrs( gp.quicksum( y[j,k] for k in ship_to2s ) <= hub[j] for j in ship_tos )

{'Los Angeles': <gurobi.Constr *Awaiting Model Update*>,
 'Philadelphia': <gurobi.Constr *Awaiting Model Update*>,
 'Phoenix': <gurobi.Constr *Awaiting Model Update*>}

In [28]:
# add demand constraints
m.addConstrs( gp.quicksum( y[j,k] for j in ship_tos ) == plants[k] for k in ship_to2s )

{'Austin': <gurobi.Constr *Awaiting Model Update*>,
 'Baltimore': <gurobi.Constr *Awaiting Model Update*>,
 'Charlotte': <gurobi.Constr *Awaiting Model Update*>,
 'Chicago': <gurobi.Constr *Awaiting Model Update*>,
 'Columbus': <gurobi.Constr *Awaiting Model Update*>}

In [29]:
m.addConstrs(gp.quicksum( x[i,j] for i in ship_froms) == gp.quicksum( y[j,k] for k in ship_to2s) for j in ship_tos)

{'Los Angeles': <gurobi.Constr *Awaiting Model Update*>,
 'Philadelphia': <gurobi.Constr *Awaiting Model Update*>,
 'Phoenix': <gurobi.Constr *Awaiting Model Update*>}

In [30]:
# solve model
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 13 rows, 21 columns and 57 nonzeros
Model fingerprint: 0xe0f71454
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 2e+03]
Presolve time: 0.01s
Presolved: 13 rows, 21 columns, 57 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.2181000e+06   6.700000e+03   0.000000e+00      0s
       8    6.0187000e+06   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.02 seconds (0.00 work units)
Optimal objective  6.018700000e+06


In [31]:
print("The total transportation cost is",m.objVal)
print("This is a cost of",m.objVal/6000)
for i in ship_froms:
    for j in ship_tos:
        print("From",i,"to",j,"ship",x[i,j].x)

The total transportation cost is 6018700.0
This is a cost of 1003.1166666666667
From texas City to Los Angeles ship 1000.0
From texas City to Philadelphia ship 1000.0
From texas City to Phoenix ship 0.0
From san antonio to Los Angeles ship 0.0
From san antonio to Philadelphia ship 0.0
From san antonio to Phoenix ship 2000.0


In [32]:
print("The total transportation cost is",m.objVal)
print("This is a cost of",m.objVal/6000)
for j in ship_tos:
    for k in ship_to2s:
        print("From",j,"to",k,"ship",y[j,k].x)

The total transportation cost is 6018700.0
This is a cost of 1003.1166666666667
From Los Angeles to Austin ship 0.0
From Los Angeles to Baltimore ship 500.0
From Los Angeles to Charlotte ship 500.0
From Los Angeles to Chicago ship 0.0
From Los Angeles to Columbus ship 0.0
From Philadelphia to Austin ship 600.0
From Philadelphia to Baltimore ship 0.0
From Philadelphia to Charlotte ship 0.0
From Philadelphia to Chicago ship 400.0
From Philadelphia to Columbus ship 0.0
From Phoenix to Austin ship 900.0
From Phoenix to Baltimore ship 800.0
From Phoenix to Charlotte ship 0.0
From Phoenix to Chicago ship 0.0
From Phoenix to Columbus ship 300.0


In [33]:
print("The total transportation cost is",m.objVal)
print("This is a cost of",m.objVal/6000)
for i in ship_froms:
    for j in ship_tos:
        if x[i,j].x > 0.5:
            print("From",i,"to",j,"ship",x[i,j].x)

The total transportation cost is 6018700.0
This is a cost of 1003.1166666666667
From texas City to Los Angeles ship 1000.0
From texas City to Philadelphia ship 1000.0
From san antonio to Phoenix ship 2000.0


In [34]:
print("The total transportation cost is",m.objVal)
print("This is a cost of",m.objVal/6000,"per drum set.")
for j in ship_tos:
    for k in ship_to2s:
        if y[j,k].x > 0.5:
            print("From",j,"to",k,"ship",y[j,k].x,"drum sets (or pallets).")

The total transportation cost is 6018700.0
This is a cost of 1003.1166666666667 per drum set.
From Los Angeles to Baltimore ship 500.0 drum sets (or pallets).
From Los Angeles to Charlotte ship 500.0 drum sets (or pallets).
From Philadelphia to Austin ship 600.0 drum sets (or pallets).
From Philadelphia to Chicago ship 400.0 drum sets (or pallets).
From Phoenix to Austin ship 900.0 drum sets (or pallets).
From Phoenix to Baltimore ship 800.0 drum sets (or pallets).
From Phoenix to Columbus ship 300.0 drum sets (or pallets).
